# Recession Predictor

Introduction
Currently in the longest expansion ever. All the talk in the news about possible recession in 2020, the recent treasury interest inversion, market corrections in 2018 & 2019, and generally low investor sentiment, etc. Recessions happen every 10 years or so, are we overdue for one?

Explain other methodologies used to predict recession.
*Federal Reserve
*Rabobank
*Blah
*Blah

Explain what I'm trying to do, not only to look at recession as a binary as NBER describes it, but also to predict future GDP in the [] futre terms.

Explain factors considered:
* Have sufficient data, if a dataset only has recent data, it cannot be used.
* Changes should lead, not trail, a recession, to act as indicators.
* 

* [10 Year Minus 3 Month Treasuries](https://fred.stlouisfed.org/series/T10Y3M)- Federal Reserve's methodology.
* Labor Participation Rate- Too macro driven, such as women entering workforce, or retirement of Baby Boomers.
* S&P 500 Debt to Equity Ratio- Hard to find on the web.
* [S&P 500](https://finance.yahoo.com/quote/%5EGSPC?p=^GSPC)- Weighted stock index of 500 companies listed on US exchanges with the largest market cap.

* [S&P 500 Cyclically Adjusted PE Ratio](https://www.quandl.com/data/MULTPL/SHILLER_PE_RATIO_MONTH-Shiller-PE-Ratio-by-Month)- 
* **S&P 500 RE Ratio**- Profit can be manipulated through operations or accounting, or changes in tax laws, such as Trump's tax cut. Ratio, however, cannot.
* Equity/Money Market Ratio- Hard to find on the web. Money market volume has been artificially depressed due to FED holding low rates.
* Personal Savings Rate- Too macro driven, such as women entering workforce, or retirement of Baby Boomers.
* Incremental Capital Outputs Ratio- Hard to find on the web. Calculates how much additional capital investment is needed to create growth. 
* [Outstanding Short Term Repo (3 Months)](http://law.emory.edu/ecgar/content/volume-5/issue-2/essays/repo-recession-financial-regulation.html)- Hard to find on the web. Before the Great Recession, investment banks used short term repo to inject liquidity to stay afloat. Uptick in short term repo may indicate credit crunch.
* [**Private Investment as Percent of GDP**](https://fred.stlouisfed.org/series/PNFI)- Investment represents expenditure on capital goods and residential properties. rovides an indicator for future productivity and GDP growth.

* [**Delinquency in consumer loans**](https://fred.stlouisfed.org/series/DRCLACBS)- 

Factors dismissed:

* S&P 500 PE Ratio- Opt for Cyclycally Adjusted PE Ratio instead.
* [Purchasing Managers Index](https://www.investopedia.com/terms/p/pmi.asp)- Locked behind paywall. A monthly survey of supply chain professionals in 19 primary industries to expansion or contraction.
* [Leading Economic Indicators Index](https://www.investopedia.com/terms/c/cili.asp)- Locked behind paywall. An index of 10 economic factors.
* Consumer Confidence Index- Locked behind paywall. Survey of consumer purchases and sentiments.
* VIX Volativity Index- Describes the volatility, not the direction, of stocks. Also does not describe economic strength.
* Wage Growth- Symptom, not sign, of economic strength. Wages also has not kept up with GDP growth.
* [**Consumer Price Index for Urban Consumers**](https://www.bls.gov/opub/btn/volume-3/why-does-bls-provide-both-the-cpi-w-and-cpi-u.htm)- Price inflation covering 88% of Americans, calculated from expenditures.
* Effective Federal funds rate- Not an organic indicator of market conditions. Different FED chairs have different doctrines. The FED was fighting inflation in the 70s, deregulation in the 80s, inflation targeting in the 90s & 2000s, then quantitative easing based on employment data in the 2010s 
* Change in working hours- Symptom, not sign, of economic strength.

## Data Preparation

First, import the necessary packages.

In [1]:
import fredapi
import requests
import pandas as pd
import json

Explain data source: Federal Reserve Economic Research (FRED), and Quandl

#### FRED Data

In [2]:
f = fredapi.Fred(api_key='8b91217446b6307d20cb5e4fcfba70eb')
# daily data
f_t10y3m = f.get_series('T10Y3M')
# monthly data
f_cpiu = f.get_series('CPIAUCSL') # assume end of period
# quarterly data
f_gdp = f.get_series('GDP')
f_gdp_growth = f.get_series('A191RL1Q225SBEA')
f_pinvest = f.get_series('PNFI') # assume end of period, need to divide by GDP to get percentage
f_delcon = f.get_series('DRCLACBS') # end of period

We need to convert FRED data from series into dataframe.

In [3]:
f_gdp = f_gdp.to_frame().reset_index()
f_gdp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294 entries, 0 to 293
Data columns (total 2 columns):
index    294 non-null datetime64[ns]
0        290 non-null float64
dtypes: datetime64[ns](1), float64(1)
memory usage: 4.7 KB


In [4]:
f_t10y3m = f_t10y3m.to_frame().reset_index()
f_cpiu = f_cpiu.to_frame().reset_index()
f_gdp_growth = f_gdp_growth.to_frame().reset_index()
f_pinvest = f_pinvest.to_frame().reset_index()
f_delcon = f_delcon.to_frame().reset_index()

In [5]:
#data = [f_t10y3m, fred_cpiu]

#### Quandl Data

In [17]:
q_sp500 = pd.DataFrame.from_dict(requests.get('https://www.quandl.com/api/v3/datasets/MULTPL/SP500_REAL_PRICE_MONTH.json?api_key=8ufKe7Y2JMsYPU3CGN7m').json()['dataset']['data'])
q_sp500_pe = pd.DataFrame.from_dict(requests.get('https://www.quandl.com/api/v3/datasets/MULTPL/SHILLER_PE_RATIO_MONTH.json?api_key=8ufKe7Y2JMsYPU3CGN7m').json()['dataset']['data'])
q_sp500_pr = pd.DataFrame.from_dict(requests.get('https://www.quandl.com/api/v3/datasets/MULTPL/SP500_PSR_QUARTER.json?api_key=8ufKe7Y2JMsYPU3CGN7m').json()['dataset']['data'])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1787 entries, 0 to 1786
Data columns (total 2 columns):
0    1787 non-null object
1    1787 non-null float64
dtypes: float64(1), object(1)
memory usage: 28.0+ KB


We need to convert object type to datetime type.

In [ ]:
pe_ratio.iloc[:,0] = pd.to_datetime(pe_ratio.iloc[:,0], format='%Y-%m-%d')
pe_ratio.info()

Since Quandl data is taken at the beginning of the month, we will subtract 1 day to get end of month. Assuming no catastrophic changes should occur in the span of 1 day.

In [ ]:
pe_ratio.iloc[:,0] = pe_ratio.iloc[:,0] - pd.to_timedelta(1, unit='day')
pe_ratio.head()

#### Steps
1. get json data and convert to dataframe
2. convert fred series into dataframe
3. convert dates into srptime type
4. convert data to same timeframe (monthly)
    *daily to monthly
    *quarterly to monthly, fill in missing data
5. find reasonable start and end time
6. remove dates with null data
7. join all data into same dataframe

Misc.
* table of contents & navigation

Code Dump
'''to view json'''
print(json.dumps(q_pr_ratio, indent=2))

'''convert json into DataFrame
q_pe_ratio = pd.DataFrame.from_dict(r['dataset']['data'])

In [ ]:
# find oldest date
# find newest date

# fred_t10y3m.head(1)
# fred_sp500.head(1)
# fred_cpiw.head(1)
# fred_gdp.head(1)
# fred_gdp_growth.head(1)
# fred_pinvest.head(1)
# fred_delcon.head(1)

## Data Analysis

## Findings

## Limitations

This predictor does not pinpoint the cause of a recession, but makes a general comment about the state of the economy. The underlying assumption is that these inputs are efficient enough to reflect long term market conditions, but inefficient enough to price in the short term fluctuations in GDP.

## Closing Thoughts